### Configuração e Otimização de Consultas no PySpark

In [0]:
# ---- 1. Inicialização da SparkSession ----
from pyspark.sql import SparkSession

# Configurações para suporte ao Delta Lake
spark = SparkSession.builder \
    .appName("Consultas Otimizadas - Leitura Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# ---- 2. Verificando o número de núcleos disponíveis ----
# Determina o número de núcleos ativos no cluster Spark
num_cores = sc._jsc.sc().getExecutorMemoryStatus().keySet().size()
print(f"Número de núcleos no cluster: {num_cores}")


#### Evidências das Tabelas Delta


In [0]:
from delta.tables import DeltaTable

# Caminho base para as tabelas Delta
delta_path = "dbfs:/mnt/lhdw/gold/vendas_delta"

# ---- 3. Carregar e visualizar Fato Vendas ----
delta_table = DeltaTable.forPath(spark, f"{delta_path}/fato_vendas")
print("Evidência da tabela Fato Vendas:")
delta_table.toDF().show()

# ---- 4. Carregar e visualizar Dimensão Produto ----
dim_produto_df = DeltaTable.forPath(spark, f"{delta_path}/dim_produto")
print("Evidência da tabela Dim Produto:")
dim_produto_df.toDF().show()

# ---- 5. Carregar e visualizar Dimensão Geografia ----
dim_geografia_df = DeltaTable.forPath(spark, f"{delta_path}/dim_geografia")
print("Evidência da tabela Dim Geografia:")
dim_geografia_df.toDF().show()

# ---- 6. Carregar e visualizar Dimensão Categoria ----
dim_categoria_df = DeltaTable.forPath(spark, f"{delta_path}/dim_categoria")
print("Evidência da tabela Dim Categoria:")
dim_categoria_df.toDF().show()

# ---- 7. Carregar e visualizar Dimensão Cliente ----
# Alternativa usando a API de leitura direta no formato Delta
dim_cliente_df = spark.read.format("delta").load(f"{delta_path}/dim_cliente")
print("Evidência da tabela Dim Cliente:")
dim_cliente_df.show()

# ---- 8. Carregar e visualizar Dimensão Fabricante ----
dim_fabricante_df = DeltaTable.forPath(spark, f"{delta_path}/dim_fabricante")
print("Evidência da tabela Dim Fabricante:")
dim_fabricante_df.toDF().show()

# ---- 9. Carregar e visualizar Dimensão Segmento ----
dim_segmento_df = DeltaTable.forPath(spark, f"{delta_path}/dim_segmento")
print("Evidência da tabela Dim Segmento:")
dim_segmento_df.toDF().show()



Evidência da tabela Fato Vendas:
+----------+----------+------------+-----------+-------------+-----------+--------+-------------+-------------+-----------+----+---+----------------+
| DataVenda|sk_produto|sk_categoria|sk_segmento|sk_fabricante| sk_cliente|Unidades|PrecoUnitario|CustoUnitario|TotalVendas| Ano|Mes|data_atualizacao|
+----------+----------+------------+-----------+-------------+-----------+--------+-------------+-------------+-----------+----+---+----------------+
|2012-04-28|        73|           2|          7|            1|34359741806|       1|       100.79|        73.58|     100.79|2012|  4|            null|
|2012-04-05|        73|           2|          7|            1|42949677547|       1|       100.79|        73.58|     100.79|2012|  4|            null|
|2012-04-29|        73|           2|          7|            1|60129543459|       1|       100.79|        73.58|     100.79|2012|  4|            null|
|2012-04-02|        73|           2|          7|            1|17179

#### Consultas Otimizadas e Boas Práticas

In [0]:
# ---- 10. Aplicar Predicate Pushdown ----
# Exemplo de otimização ao filtrar diretamente os dados durante a leitura
gold_path = "dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas"
df_filtrado = spark.read.format("delta").load(gold_path).filter("Ano = 2012 AND Mes = 10")
print("Dados filtrados por Ano e Mês (predicate pushdown):")
df_filtrado.show()

# ---- 11. Implementar Broadcast Join ----
from pyspark.sql.functions import year, sum, broadcast, desc

# Leitura da tabela fato vendas
vendas_df = spark.read.format("delta").load(f"{delta_path}/fato_vendas")
# Leitura da dimensão categoria
categoria_df = spark.read.format("delta").load(f"{delta_path}/dim_categoria")

# Otimização: usar broadcast para dimensões pequenas
categoria_df = broadcast(categoria_df)

# Realizar join otimizado
resultado_df = vendas_df.join(categoria_df, vendas_df.sk_categoria == categoria_df.sk_categoria) \
    .groupBy("Categoria", "Ano") \
    .agg(sum("TotalVendas").alias("TotalVendas")) \
    .orderBy("Ano", desc("TotalVendas"))

print("Resultados agrupados por Categoria e Ano:")
resultado_df.show()


Dados filtrados por Ano e Mês (predicate pushdown):
+----------+----------+------------+-----------+-------------+-----------+--------+-------------+-------------+-----------+----+---+----------------+
| DataVenda|sk_produto|sk_categoria|sk_segmento|sk_fabricante| sk_cliente|Unidades|PrecoUnitario|CustoUnitario|TotalVendas| Ano|Mes|data_atualizacao|
+----------+----------+------------+-----------+-------------+-----------+--------+-------------+-------------+-----------+----+---+----------------+
|2012-10-17|        19|           2|          6|            1|60129550649|       1|        36.74|        26.82|      36.74|2012| 10|            null|
|2012-10-31|        19|           2|          6|            1|       7385|       1|        36.74|        26.82|      36.74|2012| 10|            null|
|2012-10-24|        19|           2|          6|            1|25769809950|       1|        36.74|        26.82|      36.74|2012| 10|            null|
|2012-10-24|        19|           2|          6|

#### Boas Práticas de Otimização no Spark

- Predicate Pushdown: Filtros aplicados diretamente na leitura evitam carregar dados desnecessários.
- Broadcast Join: Replica dimensões pequenas em todos os nós para junções mais rápidas.
- Particionamento: Particionar a tabela Fato por colunas como Ano e Mês melhora consultas temporais.
- Evite UDFs: Use funções internas do Spark SQL para melhor performance.
- Reparticionamento: Utilize coalesce para reduzir partições sem shuffle, se necessário.
- Cache: Armazene dados frequentemente reutilizados na memória para acelerar consultas.
- Uso de Adaptive Query Execution (AQE): Ajusta dinamicamente o plano de execução com base nos dados processados.

Estas práticas ajudam a maximizar a eficiência de leitura e escrita no PySpark, reduzindo o tempo de execução e otimizando recursos computacionais.